In [25]:
data = spark.read.csv("file:/home/hduser/future-sales-data.csv", inferSchema = True)\
.toDF("txnid","txndate","amount","custid","category","product","city","state","paymenttype")
# Convert sales_date to timestamp

from pyspark.sql.functions import year, month, dayofmonth, dayofweek
from pyspark.sql.functions import unix_timestamp, col

df = data.withColumn("sales_date", unix_timestamp(col("txndate"), "MM-dd-yyyy").cast("timestamp"))

# Extract components from sales_date
df = df.withColumn("year", year(df["sales_date"]))
df = df.withColumn("month", month(df["sales_date"]))
df = df.withColumn("dayofmonth", dayofmonth(df["sales_date"]))
df = df.withColumn("dayofweek", dayofweek(df["sales_date"]))
df = df.select("year","month","dayofmonth","dayofweek","state","city","product","category")
df.show()

+----+-----+----------+---------+--------------+--------------+--------------------+--------------------+
|year|month|dayofmonth|dayofweek|         state|          city|             product|            category|
+----+-----+----------+---------+--------------+--------------+--------------------+--------------------+
|2011|    6|        26|        1|     Tennessee|   Clarksville|Cardio Machine Ac...|  Exercise & Fitness|
|2011|    5|        26|        5|    California|    Long Beach|Weightlifting Gloves|  Exercise & Fitness|
|2011|    6|         1|        4|    California|       Anaheim|Weightlifting Mac...|  Exercise & Fitness|
|2011|    6|         5|        1|     Wisconsin|     Milwaukee|    Gymnastics Rings|          Gymnastics|
|2011|   12|        17|        7|     Tennessee|   Nashville  |        Field Hockey|         Team Sports|
|2011|    2|        14|        2|      Illinois|       Chicago|Camping & Backpac...|  Outdoor Recreation|
|2011|   10|        28|        6|South Carolin

In [26]:
from pyspark.ml.pipeline import PipelineModel
loaded_model = PipelineModel.load("file:/home/hduser/SparkML-Workouts/sales_model")

In [27]:
loaded_predictions = loaded_model.transform(df)
loaded_predictions.show()

+----+-----+----------+---------+--------------+--------------+--------------------+--------------------+---------+----------+------------+-------------+----------------+---------------+-----------------+---------------+------------+---------------+---------------+-------------+--------------------+------------------+
|year|month|dayofmonth|dayofweek|         state|          city|             product|            category|cityIndex|stateIndex|productIndex|categoryIndex|         cityVec|       stateVec|       productVec|    categoryVec|     yearVec|       monthVec|  dayofmonthVec| dayofweekVec|            features|        prediction|
+----+-----+----------+---------+--------------+--------------+--------------------+--------------------+---------+----------+------------+-------------+----------------+---------------+-----------------+---------------+------------+---------------+---------------+-------------+--------------------+------------------+
|2011|    6|        26|        1|     Te